---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
<br><a href = "https://pandas.pydata.org/pandas-docs/stable/reference/frame.html">Pandas Doccummentation</a>
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('olympics.csv', index_col = 0, skiprows = 1)

# df.index
# df.columns
# Data Cleaning - Fixing row index, Renaming Columns, Eliminating unwanted data
for column in df.columns:
    # print(column, end = " ")
    if(column[0:2] == '01'):
        df.rename(columns = {column : 'Gold' + column[2:]}, inplace = True)
    if(column[0:2] == '02'):
        df.rename(columns = {column : 'Silver' + column[2:]}, inplace = True)
    if(column[0:2] == '03'):
        df.rename(columns = {column : 'Bronze' + column[2:]}, inplace = True)
    if(column[0:1] == '№'):
        df.rename(columns = {column : '#' + column[1:]}, inplace = True)

countries = df.index.str.split('\s\(')

df.index = countries.str[0]

df['Id'] = countries.str[1].str[0:3]

# First country in data frame

# df.index[0]

df = df.drop('Totals')

df.head()

,# Summer,Gold !,Silver !,Bronze !,Total,# Winter,Gold !.1,Silver !.1,Bronze !.1,Total.1,# Games,Gold !.2,Silver !.2,Bronze !.2,Combined total,Id
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [4]:
def answer_zero():
    return df.iloc[0]
answer_zero()

# Summer           13
Gold !              0
Silver !            0
Bronze !            2
Total               2
# Winter            0
Gold !.1            0
Silver !.1          0
Bronze !.1          0
Total.1             0
# Games            13
Gold !.2            0
Silver !.2          0
Bronze !.2          2
Combined total      2
Id                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [5]:
def answer_one():
    maxGold = max(df['Gold !'])
    res = df[df['Gold !'] == maxGold].index[0]
    return res
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [6]:
def answer_two():
    cpy_df = df.copy()
    cpy_df['diff'] = cpy_df['Gold !'] - cpy_df['Gold !.1']
    maxDiff = max(cpy_df['diff'])
    res = cpy_df[cpy_df['diff'] == maxDiff].index[0]
    return res
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [7]:
def answer_three():
    cpy_df = df.copy()
    cpy_df = cpy_df.where(df['Gold !']>0)
    cpy_df = cpy_df.where(df['Gold !.1']>0)
    cpy_df['diff'] = cpy_df['Gold !'] - cpy_df['Gold !.1']
    cpy_df['Gold_Ratio'] = cpy_df['diff']/(cpy_df['Gold !']+cpy_df['Gold !.1'])
    cpy_df_final = cpy_df[np.isfinite(cpy_df['Gold_Ratio'])]
    maxRatio = max(cpy_df_final['Gold_Ratio'])
    res = cpy_df_final[cpy_df_final['Gold_Ratio'] == maxRatio].index[0]
    return res
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [9]:
def answer_four():
    cpy_df = df.copy()
    cpy_df['Points'] = cpy_df['Gold !.2'] * 3 + cpy_df['Silver !.2'] *2 + cpy_df['Bronze !.2']
    return pd.Series(cpy_df['Points'])
answer_four()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: Points, Length: 146, dtype: int64

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [11]:
census_df = pd.read_csv('dataSets/census.csv')
census_df.head()

FileNotFoundError: [Errno 2] File b'dataSets/census.csv' does not exist: b'dataSets/census.csv'

In [10]:
# idxmax - Return index of first occurrence of maximum over requested axis.
def answer_five():
    cnts_df = census_df[census_df['SUMLEV'] == 50]
    res = cnts_df.groupby('STNAME').count()['CTYNAME']
    return res.idxmax()
answer_five()

NameError: name 'census_df' is not defined

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [ ]:
def answer_six():
    cnts_df = census_df[census_df['SUMLEV'] == 50]
    top_cnts_df = cnts_df.sort_values(by = 'CENSUS2010POP', ascending = False).groupby('STNAME').head(3)
    top_cnts_df = top_cnts_df.groupby('STNAME').sum().sort_values(by = 'CENSUS2010POP', ascending = False).head(3)
    res = top_cnts_df.index
    return list(res)
answer_six()

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [ ]:
def answer_seven():
    cnts_df = census_df[census_df['SUMLEV'] == 50]
    cnts_df = cnts_df.set_index('CTYNAME')
    columns = ['POPESTIMATE2010',
              'POPESTIMATE2011',
              'POPESTIMATE2012',
              'POPESTIMATE2013',
              'POPESTIMATE2014',
              'POPESTIMATE2015']
    maxValue = cnts_df[columns].max(axis = 1)
    minValue = cnts_df[columns].min(axis = 1)
    cnts_df['diff'] = maxValue - minValue
    res = cnts_df['diff'].idxmax()
    return res
answer_seven()

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [ ]:
def answer_eight():
    cnts_df = census_df[census_df['SUMLEV'] == 50]
    cnts_df = cnts_df[((cnts_df['REGION'] == 1) | (cnts_df['REGION'] == 2))]
    cnts_df = cnts_df[cnts_df['CTYNAME'] == 'Washington County']
    cnts_df = cnts_df[cnts_df['POPESTIMATE2015'] > cnts_df['POPESTIMATE2014']]
    return cnts_df[['STNAME','CTYNAME']]
answer_eight()